In [1]:
import agents_env
from agents_env.agent_mimic_train import HumanoidEnvTrain
from utils.SimpleConverter import SimpleConverter
from utils.util_data import *
from copy import deepcopy

In [2]:
from datetime import datetime
import functools
from IPython.display import HTML
import jax
from jax import numpy as jp
import numpy as np
from typing import Any, Dict, Sequence, Tuple, Union
from brax import base
from brax import envs
from brax import math
from brax.base import Base, Motion, Transform
from brax.envs.base import Env, PipelineEnv, State
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import train as ppo
from brax.training.agents.ppo import networks as ppo_networks
from brax.io import html, mjcf, model
from etils import epath
from flax import struct
from matplotlib import pyplot as plt
import mediapy as media
from ml_collections import config_dict
import mujoco
from mujoco import mjx
from jax import vmap
import jax.random
from jax import lax

In [3]:
#this is just dummy data to initialize the agent
trajectory = SimpleConverter('motions/humanoid3d_punch.txt')
trajectory.load_mocap()
model_path = 'models/final_humanoid.xml'

data_mocap_matrix = jp.asarray(trajectory.data)
data_pos_mocap = jp.asarray(trajectory.data_pos)
data_vel_mocap = jp.asarray(trajectory.data_vel)

data_dict_mocap = trajectory.duration_dict

data_xpos_mocap=jp.asarray(trajectory.data_xpos)
data_xrot_mocap=jp.asarray(trajectory.data_xrot)

In [4]:
#get th kp and kd for the agent
kp,kd = generate_kp_kd_gains()
print(kp)
print(kd)

[1000 1000 1000  100  100  100  400  400  400  300  400  400  400  300
  500  500  500  500  400  400  400  500  500  500  500  400  400  400]
[100 100 100  10  10  10  40  40  40  30  40  40  40  30  50  50  50  50
  40  40  40  50  50  50  50  40  40  40]


In [5]:
import yaml
from box import Box
# Path to your YAML file
yaml_file_path = 'config_params/punch.yaml'
# Load the YAML file
with open(yaml_file_path, 'r') as file:
    args = Box(yaml.safe_load(file))

In [7]:
envs.register_environment('humanoidEnvMimic',HumanoidEnvTrain)
env_name = 'humanoidEnvMimic'
env = envs.get_environment(env_name=env_name,
                           reference_trajectory_qpos=data_pos_mocap,
                           reference_trajectory_qvel = data_vel_mocap,
                            duration_trajectory=trajectory.total_time,
                            dict_duration= data_dict_mocap,
                           model_path=model_path,
                           kp_gains = kp,
                           kd_gains = kd,
                           reference_x_pos=data_xpos_mocap,
                           reference_x_rot=data_xrot_mocap,
                           args=args)
jit_reset = jax.jit(env.reset)
jit_step = jax.jit(env.step_custom)


#select the pd_control
from agents_env.pds_controllers_agents import stable_pd_controller_custom_trajectory, stable_pd_controller_trajectory,stable_pd_controller_action,feedback_pd_controller
#env.set_pd_callback(stable_pd_controller_custom_trajectory)
env.set_pd_callback(feedback_pd_controller)
#env.set_pd_callback(feedback_pd_controller)

In [8]:
env.rollout_lenght

65

In [9]:
# initialize the state
state = jit_reset(jax.random.PRNGKey(0))
rollout = [state.pipeline_state]
# grab a 500 steps
n_steps = 500

rot(array(13, dtype=int32), array(4, dtype=int32))
vel[[-2.8818053e-01  8.7779284e-02  8.1000075e-02]
 [-3.4689954e-01  7.7036172e-02  8.1654593e-02]
 [-4.1608381e-01  5.1521216e-02  8.6229317e-02]
 [-4.4027799e-01  1.7190793e-01  1.0165399e-01]
 [-4.9120674e-01  3.4506398e-01  1.5201633e-01]
 [-3.7631401e-01 -6.6355929e-02  6.9428891e-02]
 [-2.1253726e-01 -1.1931123e-01 -1.3627857e-04]
 [-3.0948627e-01  1.5163174e-01  1.0134506e-01]
 [-4.1612828e-01  2.4467927e-01  1.4186221e-01]
 [-3.4511092e-01  4.5810035e-01  3.3873204e-02]
 [-2.6687473e-01  2.3926817e-02  6.0655080e-02]
 [-2.9612988e-01  3.8792977e-01  4.8879310e-02]
 [-6.9739127e-01  7.6306152e-01 -1.9804212e-01]]
ang[[ 0.05192349 -0.23546423  0.79337656]
 [ 0.07098626 -0.07190083  0.6725171 ]
 [-0.03619116 -0.12164814  0.4629038 ]
 [ 0.3912696  -0.09505065  0.72247386]
 [ 0.39010382 -0.10333263  0.74169236]
 [ 0.15336552 -0.30837145  0.5958098 ]
 [ 0.13275181 -0.29521644  0.59468627]
 [ 0.19098729  0.08959711  0.2969229 ]
 [ 0.5

In [ ]:

for i in range(env.rollout_lenght):
    
    print(i)
    ctrl = -0.1 * jp.ones(env.sys.nu)
    #time
    time = state.pipeline_state.time
    
    #print('time: ',state.pipeline_state.time)    
         
    state = jit_step(state, ctrl)
    
    print("Is done", state.done)
    print("Is rewards", state.reward)
    
    rollout.append(state.pipeline_state)

In [ ]:
HTML(html.render(env.sys.replace(dt=env.dt), rollout))